In [1]:
#Bibliotecas
source('utils.r',local=TRUE)
library(MASS)
library(car)
library(statmod)

Loading required package: lattice
Loading required package: ggplot2
Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: Matrix
Loading required package: foreach
Loaded glmnet 2.0-13

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following object is masked from ‘package:glmnet’:

    auc

The following objects are masked from ‘package:stats’:

    cov, smooth, var



In [2]:
#Leitura do dataset já tratado
data_frame = read_dataset()

#Armazenando dados de treino e teste
train = data_frame$train
test = data_frame$test

In [3]:
#IREMOS UTILIZAR OS MODELOS FINAIS DA ETAPA DE MODELAGEM

#Modelos alternativos
alternative1 = glm(formula = default ~ t_emprego + t_endereco + div + renda + t_emprego:renda, family = binomial(link = logit), data = train)
alternative2 = glm(formula = default ~ t_emprego + t_endereco + div + renda + div:renda, family = binomial(link = logit), data = train)

#Criando matriz com os fatores expandidos
x = model.matrix(default ~ ., data = train)[, -1]
y <- ifelse(train$default == 0, 1, 0)

#Modelos de regularizacao
lasso <- cv.glmnet(x, y, family = 'binomial', alpha = 1, nfolds = 10)
ridge <- cv.glmnet(x, y, family = 'binomial', alpha = 0, nfolds = 10)

#Knn
ctrl <- trainControl(method="repeatedcv",repeats = 2)
knn <- train(default ~ ., data = train, method = "knn", trControl = ctrl, tuneLength = 5)

In [4]:
#AVALIAÇÃO DOS MODELOS VIA MÉTRICAS DA MATRIZ DE CONFUSÃO, AIC E AUC

#Matriz de confusao e metricas
confMat = confusion_matrix(alternative1,test,type='glm')
metric1 = metric_confusion_matrix(confMat)

confMat = confusion_matrix(alternative2,test,type='glm')
metric2 = metric_confusion_matrix(confMat)

confMat = confusion_matrix(lasso,test,type='reg')
metric3 = metric_confusion_matrix(confMat)

confMat = confusion_matrix(ridge,test,type='reg')
metric4 = metric_confusion_matrix(confMat)

confMat = confusion_matrix(knn,test,type='knn')
metric5 = metric_confusion_matrix(confMat)

#Metricas da matriz de confusao
Acuracia = c(metric1$accuracy,metric2$accuracy,metric3$accuracy,metric4$accuracy,metric5$accuracy)
Sensibilidade = c(metric1$sens,metric2$sens,metric3$sens,metric4$sens,metric5$sens)
Especificidade = c(metric1$espec,metric2$espec,metric3$espec,metric4$espec,metric5$espec)

#AIC dos modelos
AIC = c(extractAIC(alternative1)[2],extractAIC(alternative2)[2])
AIC = round(AIC,3)
AIC[3:5] = '-'

#AUC dos modelos
AUC = c(extract_auc(alternative1,test,type='glm'),extract_auc(alternative2,test,type='glm'))
AUC = c(AUC,extract_auc(lasso,test,type='reg'))
AUC = c(AUC,extract_auc(ridge,test,type='reg'))
AUC = c(AUC,extract_auc(knn,test,type='knn'))
AUC = round(AUC,3)

#Data frame com o resultado geral
Modelo = c('alternative1','alternative2','lasso','ridge','knn')

#Gerando dataframe
resume = data.frame(Modelo,Acuracia,Sensibilidade,Especificidade,AIC,AUC)
resume

#Note pelo data.frame que todos os modelos possuem resultados similares
#Como não é possível apenas com isso identificar o modelo apropriado e
#como pode haver variação nas métricas dado a escolha do conjunto de treino/teste
#utilizado, iremos realizar esse procedimento 50 vezes e após isso retirar
#uma média das medidas AUC,Sensibilidade e Especificidade

Modelo,Acuracia,Sensibilidade,Especificidade,AIC,AUC
alternative1,0.790,0.811,0.533,297.894,80.865
alternative2,0.785,0.805,0.533,294.226,81.622
lasso,0.785,0.805,0.533,-,80.180
ridge,0.785,0.811,0.467,-,80.180
knn,0.790,0.805,0.600,-,79.838


In [5]:
#AVALIAÇÃO DA MÉDIA DE AUC, Sensibilidade e Especificidade

#Leitura do dataset já tratado
data_frame = read_dataset(split=FALSE)

#Vetores que conteram as medias dos AUC's, Sensisibilidade e Especificidade
meanAUC1 = meanAUC2 = meanAUC3 = meanAUC4 = meanAUC5 = c()
meanSENS1 = meanSENS2 = meanSENS3 = meanSENS4 = meanSENS5 = c()
meanESP1 = meanESP2 = meanESP3 = meanESP4 = meanESP5 = c()


for (i in 1:50){
    data = samples(data_frame)
    train = data$train
    test = data$test
    
    #Modelos alternativos
    alternative1 = glm(formula = default ~ t_emprego + t_endereco + div + renda + t_emprego:renda, family = binomial(link = logit), data = train)
    alternative2 = glm(formula = default ~ t_emprego + t_endereco + div + renda + div:renda, family = binomial(link = logit), data = train)
    
    #Criando matriz com os fatores expandidos
    x = model.matrix(default ~ ., data = train)[, -1]
    y <- ifelse(train$default == 0, 1, 0)

    #Modelos de regularizacao
    lasso <- cv.glmnet(x, y, family = 'binomial', alpha = 1, nfolds = 10)
    ridge <- cv.glmnet(x, y, family = 'binomial', alpha = 0, nfolds = 10)

    #Knn
    ctrl <- trainControl(method="repeatedcv",repeats = 2)
    knn <- train(default ~ ., data = train, method = "knn", trControl = ctrl, tuneLength = 5)    
    
    #Media dos AUC's
    meanAUC1 = c(meanAUC1,extract_auc(alternative1,test,type='glm'))
    meanAUC2 = c(meanAUC2,extract_auc(alternative2,test,type='glm'))
    meanAUC3 = c(meanAUC3,extract_auc(lasso,test,type='reg'))
    meanAUC4 = c(meanAUC4,extract_auc(ridge,test,type='reg'))
    meanAUC5 = c(meanAUC5,extract_auc(knn,test,type='knn'))
        
    #Métricas da matriz
    confMat = confusion_matrix(alternative1,test,type='glm')
    metric1 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(alternative2,test,type='glm')
    metric2 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(lasso,test,type='reg')
    metric3 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(ridge,test,type='reg')
    metric4 = metric_confusion_matrix(confMat)

    confMat = confusion_matrix(knn,test,type='knn')
    metric5 = metric_confusion_matrix(confMat)
    
    #Media da sensibilidade e especificidade
    meanSENS1 = c(meanSENS1,metric1$sens)
    meanSENS2 = c(meanSENS2,metric2$sens)
    meanSENS3 = c(meanSENS3,metric3$sens)
    meanSENS4 = c(meanSENS4,metric4$sens)
    meanSENS5 = c(meanSENS5,metric5$sens)
    
    meanESP1 = c(meanESP1,metric1$espec)
    meanESP2 = c(meanESP2,metric2$espec)
    meanESP3 = c(meanESP3,metric3$espec)
    meanESP4 = c(meanESP4,metric4$espec)
    meanESP5 = c(meanESP5,metric5$espec)
}

In [6]:
#Data frame com resultado final
AUC = cbind(mean(meanAUC1),mean(meanAUC2),mean(meanAUC3),mean(meanAUC4),mean(meanAUC5))
SEN = cbind(mean(meanSENS1),mean(meanSENS2),mean(meanSENS3),mean(meanSENS4),mean(meanSENS5))
ESP = cbind(mean(meanESP1),mean(meanESP2),mean(meanESP3),mean(meanESP4),mean(meanESP5))
resume1 = data.frame(AUC=AUC,row.names = c('AUC'))
names(resume1) = c('alternative1','alternative2','lasso','ridge','knn')
resume2 = data.frame(SEN,row.names=c('SEN'))
names(resume2) = c('alternative1','alternative2','lasso','ridge','knn')
resume3 = data.frame(ESP,row.names=c('ESP'))
names(resume3) = c('alternative1','alternative2','lasso','ridge','knn')
resume = round(rbind(resume1,resume2,resume3),3)
resume

#Note pelo data.frame que a média AUC acaba sendo muito similar nos modelos lineares generalizados.
#O Knn possui um desempenho um pouco inferior. Com relação a sensibilidade, ela também é similar 
#em todos os modelos. Por fim, note que são os modelos alternativos que possuem uma maior especificidade
#Logo, podemos concluir que estes serão os modelos mais adequados a serem utilizados na classificação e sua
#escolha dependerá dos tomadores de decisão. Isso porque como os classificadores possuem parâmetro diferentes,
#serão eles quem decidirão quais covariáveis serão mais interessantes para o seu problema.

,alternative1,alternative2,lasso,ridge,knn
AUC,81.879,82.460,80.823,80.984,76.908
SEN,0.819,0.814,0.826,0.831,0.796
ESP,0.632,0.657,0.589,0.587,0.537
